<a href="https://colab.research.google.com/github/Ali-mohammadi-design/Hugging_Face/blob/main/How_LLM_works_Tokenization_and_Probabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the LLM:


Encoder: Words-----> tokens----> codes (numbers)----> Vector Embed


Decoders: Vector embed---> tokens with possibilities ----> most possible word

Note: Every token has a unique identification number!

In [68]:
import torch

In [69]:
torch.__version__

'2.3.1+cu121'

In [70]:
import transformers

In [71]:
transformers.__version__

'4.42.4'

In [72]:
from transformers import AutoTokenizer

Note: Auto tokenizer would convert words in sentences to tokens.

Here we would use gpt2 tokenizer.

we use autotokenizer to download gpt2 tokenizer

In [73]:
tokenizer= AutoTokenizer.from_pretrained("gpt2")

In [182]:
sentence=" what is the capital of the france?"

In [183]:
input_ids=tokenizer(sentence, return_tensors='pt')
#pt means pytorch tensors, because here we are working with pytorch

In [184]:
input_ids

{'input_ids': tensor([[ 644,  318,  262, 3139,  286,  262, 1216,  590,   30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [185]:
input_ids=tokenizer(sentence, return_tensors='pt').input_ids

In [186]:
input_ids

tensor([[ 644,  318,  262, 3139,  286,  262, 1216,  590,   30]])

In [187]:
tokenizer.decode(644)

' what'

In [188]:
tokenizer.decode(262)

' the'

In [189]:
for token_id in input_ids[0]:
  print(tokenizer.decode(token_id))

 what
 is
 the
 capital
 of
 the
 fr
ance
?


In [207]:
sentence2= "Hello, How are "

In [191]:
input_ids2=tokenizer(sentence2, return_tensors='pt').input_ids

In [208]:
for token_id in input_ids2[0]:
  print(tokenizer.decode(token_id))

Hello
,
 How
 are
 


How the LLM select the next words?

In [209]:
from transformers import AutoModelForCausalLM

AutoModelForCausalLM is used to load models that are specifically designed for tasks where the goal is to predict the next token in a sequence. This is a common approach in language modeling and text generation

In [210]:
gpt2=AutoModelForCausalLM.from_pretrained("gpt2")

In [211]:
outputs=gpt2(input_ids2)

In [212]:
outputs.logits

tensor([[[ -35.2362,  -35.3266,  -38.9753,  ...,  -44.4645,  -43.9974,
           -36.4580],
         [-112.6171, -114.5832, -116.5725,  ..., -119.0128, -118.8059,
          -111.6917],
         [-105.9645, -107.2160, -111.5645,  ..., -117.1641, -117.2828,
          -108.2401],
         [ -83.8546,  -85.1004,  -88.2035,  ...,  -89.0557,  -92.7577,
           -85.3932],
         [ -63.2392,  -63.8590,  -65.6387,  ...,  -72.1295,  -71.1212,
           -67.4067]]], grad_fn=<UnsafeViewBackward0>)

Note: For each token in this sentence we have a list of possible tokens that could be located after that token.
Normally we select the last token and analyze its probabilities.

In [213]:
final_logits=gpt2(input_ids2).logits[0,-1]
#we choose the very last word in the logits

In [214]:
final_logits

tensor([-63.2392, -63.8590, -65.6387,  ..., -72.1295, -71.1212, -67.4067],
       grad_fn=<SelectBackward0>)

These are the predictions of the next word after the last token

In [215]:
final_logits.argmax()
#we get the index location logit of the most probable token
#token_id <---> index location logit

tensor(1849)

argmax(): The argmax() function identifies the position (index) of the maximum value in an array or tensor.

In [216]:
tokenizer.decode(final_logits.argmax())

'\xa0'

In [217]:
top10_logits=torch.topk(final_logits,10)

In [201]:
for index in top10_logits.indices:
  print(tokenizer.decode(index))

 
_____
____
________
_______
【
????
ids
~~
------------------------


In [202]:
top10=torch.topk(final_logits.softmax(dim=0),10)

In [203]:
top10

torch.return_types.topk(
values=tensor([0.1959, 0.1006, 0.0891, 0.0737, 0.0362, 0.0275, 0.0235, 0.0203, 0.0178,
        0.0173], grad_fn=<TopkBackward0>),
indices=tensor([ 1849, 29343,  1427,  2602, 37405, 31854,  9805,  2340,  4907, 22369]))

In [206]:
for value, index in zip(top10.values, top10.indices):
  print(f"{tokenizer.decode(index)}---{value.item()}")

 ---0.19591133296489716
_____---0.10057950764894485
____---0.08908674120903015
________---0.07372939586639404
_______---0.03624147176742554
【---0.02751830965280533
????---0.02348994091153145
ids---0.02031140774488449
~~---0.0177523884922266
---------------------------0.01730778068304062
